In [47]:
import pandas as pd
import requests
import numpy as np
from datetime import datetime, timedelta

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 1000)

In [36]:
hubs = ['ALBE', 'ALDE', 'AT', 'BE', 'CZ', 'DE_LU', 'FR', 'HR', 'HU', 'NL', 'PL', 'RO', 'SI', 'SK']

start = datetime(2022, 9, 1).strftime('%Y-%m-%d')
#end = datetime(2022, 9, 2).strftime('%Y-%m-%d')
end = datetime(2023, 3, 31).strftime('%Y-%m-%d')

In [32]:
def retrieve_ptdf(fromDate, toDate):
    data = {
        'fromUtc': f'{fromDate}T00:00:00.000Z',
        'toUtc': f'{toDate}T23:00:00.000Z',
    }

    try:
        response = requests.post('https://publicationtool.jao.eu/core/api/data/preFinalComputation', json=data)
        response.raise_for_status()
    except requests.exceptions.HTTPError as err:
        print(err)
   
    return pd.DataFrame(response.json()['data'])

In [33]:
df_grid = pd.read_excel('../flow_based_domain/grid.xlsx', index_col=0)

In [38]:
# ZONAL PTDF and RAM
df_ptdf = retrieve_ptdf(start, end)

df_ptdf = df_ptdf.rename(columns={
    'dateTimeUtc': 'DateTime'
})
df_ptdf = df_ptdf.set_index('DateTime')
df_ptdf.index = pd.to_datetime(df_ptdf.index)
df_ptdf = df_ptdf.tz_localize(None)
df_ptdf = df_ptdf.reset_index()

df_ptdf = df_ptdf[df_ptdf.cneEic != 'NA']
df_ptdf = df_ptdf[[
    'DateTime', 'cneName', 'cneEic', 'tso', 'direction', 'substationFrom', 'substationTo', 'contingencies', 'ram', 'ptdf_ALBE', 'ptdf_ALDE', 'ptdf_AT', 'ptdf_BE', 'ptdf_CZ', 'ptdf_DE', 'ptdf_FR', 'ptdf_HR', 'ptdf_HU', 'ptdf_NL', 
    'ptdf_PL', 'ptdf_RO', 'ptdf_SI', 'ptdf_SK'
]]

display(df_ptdf)
#df_ptdf.to_excel("ptdf_z_raw.xlsx")

,DateTime,cneName,cneEic,tso,direction,substationFrom,substationTo,contingencies,ram,ptdf_ALBE,ptdf_ALDE,ptdf_AT,ptdf_BE,ptdf_CZ,ptdf_DE,ptdf_FR,ptdf_HR,ptdf_HU,ptdf_NL,ptdf_PL,ptdf_RO,ptdf_SI,ptdf_SK
0,2022-09-01 00:00:00,Ernsthofen 2 - Hausruck 203C,14T-220-0-0203CD,APG,DIRECT,Ernsthofen 2,Hausruck,"[{'number': 1, 'branchName': 'St. Peter 2 - St...",439.0,-0.06186,-0.06238,-0.06499,-0.06181,-0.02490,-0.06303,-0.06025,-0.00273,0.00340,-0.06144,-0.02183,0.00010,-0.00259,-0.00200
1,2022-09-01 00:00:00,Ernsthofen 2 - Hausruck 203C,14T-220-0-0203CD,APG,OPPOSITE,Ernsthofen 2,Hausruck,"[{'number': 1, 'branchName': 'St. Peter 2 - St...",341.0,0.06186,0.06238,0.06499,0.06181,0.02490,0.06303,0.06025,0.00273,-0.00340,0.06144,0.02183,-0.00010,0.00259,0.00200
2,2022-09-01 00:00:00,Wien Suedost - Gyoer 245,10T-AT-HU-00002U,APG,DIRECT,Wien Suedost,Gyoer 245,"[{'number': 1, 'branchName': 'Gyor - Neusiedl'...",146.0,0.02502,0.02502,0.03672,0.02504,0.02301,0.02519,0.02522,0.01195,-0.01617,0.02482,0.01430,-0.00085,0.02081,0.00497
3,2022-09-01 00:00:00,Neusiedl - Gyoer 246B,10T-AT-HU-00001W,APG,DIRECT,Neusiedl,Gyoer 246B,"[{'number': 1, 'branchName': 'Gyor - Wien', 'b...",155.0,0.02473,0.02472,0.03740,0.02476,0.02248,0.02487,0.02501,0.01253,-0.01655,0.02451,0.01386,-0.00089,0.02190,0.00451
4,2022-09-01 00:00:00,Neusiedl - Gyoer 246B,10T-AT-HU-00001W,APG,OPPOSITE,Neusiedl,Gyoer 246B,"[{'number': 1, 'branchName': 'Gyor - Wien', 'b...",267.0,-0.02473,-0.02472,-0.03740,-0.02476,-0.02248,-0.02487,-0.02501,-0.01253,0.01655,-0.02451,-0.01386,0.00089,-0.02190,-0.00451
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
541476,2023-03-31 22:00:00,Krizovany - Sokolnice,10T-CZ-SK-00003U,SEPS,DIRECT,Krizovany,Sokolnice,"[{'number': 1, 'branchName': 'Sokolnice - Stup...",1421.0,-0.12423,-0.12522,-0.10793,-0.12392,-0.18969,-0.12661,-0.12154,-0.03307,0.01876,-0.12507,-0.07989,0.00276,-0.06246,0.06681
541477,2023-03-31 22:00:00,Sokolnice - Stupava,10T-CZ-SK-00005Q,SEPS,OPPOSITE,Stupava,Sokolnice,"[{'number': 1, 'branchName': 'Krizovany - Soko...",1004.0,0.11961,0.12068,0.09994,0.11927,0.18129,0.12210,0.11665,0.02803,-0.02225,0.12060,0.08492,-0.00214,0.05525,-0.07919
541478,2023-03-31 22:00:00,V.Kapusany - Mukachevo (WPS),10T-SK-UA-00001W,SEPS,OPPOSITE,Velke Kapusany,Mukachevo,"[{'number': 1, 'branchName': 'R.Sobota - Sajoi...",1388.0,-0.21769,-0.21902,-0.19030,-0.21719,-0.21745,-0.22024,-0.21368,-0.11134,-0.09501,-0.21954,-0.29591,0.01699,-0.15569,-0.28346
541479,2023-03-31 22:00:00,Nosovice - Varin,10T-CZ-SK-00002W,SEPS,OPPOSITE,Nosovice,Varin,"[{'number': 1, 'branchName': 'Krizovany - Soko...",1179.0,0.15186,0.15387,0.09884,0.15112,0.15771,0.15581,0.14591,0.03331,-0.00911,0.15457,0.19936,-0.00316,0.05968,-0.06298


In [48]:
df_ptdf.columns = ['DateTime', 'name', 'eic', 'tso', 'direction', 'substation_from', 'substation_to', 'contingencies', 'ram'] + hubs

df_ptdf_test = df_ptdf.copy()
df_ptdf_test['contingencies'] = df_ptdf_test['contingencies'].apply(lambda c_list: list(filter(lambda c: c['elementType'] in ['Line', 'TieLine'], c_list)))
df_ptdf_test['contingencies_count'] = df_ptdf_test['contingencies'].apply(lambda c_list: len(c_list))

display(df_ptdf_test[['ram', 'eic', 'tso', 'contingencies_count']].groupby(['eic', 'tso', 'contingencies_count']).count())

#display(df_ptdf_test)

ram
eic              tso            contingencies_count       
10T-AT-CH-00002V APG            0                     1086
                                1                      338
10T-AT-CH-00003T APG            0                        3
10T-AT-CH-00004R APG            0                     1221
                                1                       10
10T-AT-CZ-00003X APG            0                     3362
                                1                       36
                                2                      570
10T-AT-CZ-00004V APG            1                       54
                 CEPS           0                     1539
                                2                      312
10T-AT-DE-00001B APG            0                      421
                                3                       39
10T-AT-DE-000029 APG            0                        1
10T-AT-DE-000037 APG            0                      507
                                1                     4836
                                2                      134
                                3                      169
                 TENNETGMBH     0                      139
                                1                        7
                                3                        2
10T-AT-DE-000045 APG            0                      236
                                3                       17
10T-AT-DE-000053 APG            0                       40
                                2                       53
                 TRANSNETBW     0                       64
                                2                        8
10T-AT-DE-000061 TRANSNETBW     0                      401
                                1                        1
                                2                      245
10T-AT-DE-00019T TENNETGMBH     1                        4
10T-AT-DE-000231 APG            0                      343
                                1                     1460
                 TRANSNETBW     0                      166
                                1                      517
10T-AT-DE-00026W APG            0                        2
                                1                        2
10T-AT-DE-10007U AMPRION        0                      113
                                1                        7
                                2                        2
10T-AT-DE-10024U APG            0                        1
10T-AT-HU-00001W APG            0                     2916
                                2                     1221
                 MAVIR          0                      912
                                1                        4
                                2                      638
10T-AT-HU-00002U APG            0                     1823
                                1                      335
                                2                      661
                 MAVIR          0                      432
                                1                        4
                                2                       86
10T-AT-HU-00003S APG            0                      484
                                1                      199
                                2                        1
                 MAVIR          0                     5608
                                1                     4441
10T-AT-HU-00004Q APG            0                        8
10T-AT-SI-00001T APG            0                      700
                                2                      143
                 ELES           0                     3410
                                2                      775
10T-AT-SI-00002R ELES           0                        4
10T-AT-SI-00003P APG            0                     1844
                                1                    13540
                                2                      631
                 ELES           0          

,DateTime,name,eic,tso,direction,substation_from,substation_to,contingencies,ram,ALBE,ALDE,AT,BE,CZ,DE_LU,FR,HR,HU,NL,PL,RO,SI,SK,contingencies_count
0,2022-09-01 00:00:00,Ernsthofen 2 - Hausruck 203C,14T-220-0-0203CD,APG,DIRECT,Ernsthofen 2,Hausruck,[],439.0,-0.06186,-0.06238,-0.06499,-0.06181,-0.02490,-0.06303,-0.06025,-0.00273,0.00340,-0.06144,-0.02183,0.00010,-0.00259,-0.00200,0
1,2022-09-01 00:00:00,Ernsthofen 2 - Hausruck 203C,14T-220-0-0203CD,APG,OPPOSITE,Ernsthofen 2,Hausruck,[],341.0,0.06186,0.06238,0.06499,0.06181,0.02490,0.06303,0.06025,0.00273,-0.00340,0.06144,0.02183,-0.00010,0.00259,0.00200,0
2,2022-09-01 00:00:00,Wien Suedost - Gyoer 245,10T-AT-HU-00002U,APG,DIRECT,Wien Suedost,Gyoer 245,[],146.0,0.02502,0.02502,0.03672,0.02504,0.02301,0.02519,0.02522,0.01195,-0.01617,0.02482,0.01430,-0.00085,0.02081,0.00497,0
3,2022-09-01 00:00:00,Neusiedl - Gyoer 246B,10T-AT-HU-00001W,APG,DIRECT,Neusiedl,Gyoer 246B,[],155.0,0.02473,0.02472,0.03740,0.02476,0.02248,0.02487,0.02501,0.01253,-0.01655,0.02451,0.01386,-0.00089,0.02190,0.00451,0
4,2022-09-01 00:00:00,Neusiedl - Gyoer 246B,10T-AT-HU-00001W,APG,OPPOSITE,Neusiedl,Gyoer 246B,[],267.0,-0.02473,-0.02472,-0.03740,-0.02476,-0.02248,-0.02487,-0.02501,-0.01253,0.01655,-0.02451,-0.01386,0.00089,-0.02190,-0.00451,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
541476,2023-03-31 22:00:00,Krizovany - Sokolnice,10T-CZ-SK-00003U,SEPS,DIRECT,Krizovany,Sokolnice,"[{'number': 1, 'branchName': 'Sokolnice - Stup...",1421.0,-0.12423,-0.12522,-0.10793,-0.12392,-0.18969,-0.12661,-0.12154,-0.03307,0.01876,-0.12507,-0.07989,0.00276,-0.06246,0.06681,2
541477,2023-03-31 22:00:00,Sokolnice - Stupava,10T-CZ-SK-00005Q,SEPS,OPPOSITE,Stupava,Sokolnice,"[{'number': 1, 'branchName': 'Krizovany - Soko...",1004.0,0.11961,0.12068,0.09994,0.11927,0.18129,0.12210,0.11665,0.02803,-0.02225,0.12060,0.08492,-0.00214,0.05525,-0.07919,2
541478,2023-03-31 22:00:00,V.Kapusany - Mukachevo (WPS),10T-SK-UA-00001W,SEPS,OPPOSITE,Velke Kapusany,Mukachevo,"[{'number': 1, 'branchName': 'R.Sobota - Sajoi...",1388.0,-0.21769,-0.21902,-0.19030,-0.21719,-0.21745,-0.22024,-0.21368,-0.11134,-0.09501,-0.21954,-0.29591,0.01699,-0.15569,-0.28346,2
541479,2023-03-31 22:00:00,Nosovice - Varin,10T-CZ-SK-00002W,SEPS,OPPOSITE,Nosovice,Varin,"[{'number': 1, 'branchName': 'Krizovany - Soko...",1179.0,0.15186,0.15387,0.09884,0.15112,0.15771,0.15581,0.14591,0.03331,-0.00911,0.15457,0.19936,-0.00316,0.05968,-0.06298,2


In [22]:
df_ptdf = df_ptdf[df_ptdf.direction != 'NA']

all_eics = df_ptdf.eic.unique()
unique_eics = []
invalid_obs_eics = []
non_unique_eics = []

for eic in all_eics:
    num_cne = len(df_grid[df_grid.eic == eic])
    if num_cne == 0:
        invalid_obs_eics.append(eic)
    elif num_cne == 1:
        unique_eics.append(eic)
    else:
        non_unique_eics.append(eic)
        
print('Total:', len(all_eics))
print('Unique:', len(unique_eics))
print('Non-valid observed eics:', len(invalid_obs_eics))

df_ptdf_adj = df_ptdf[~df_ptdf.eic.isin(invalid_obs_eics)]
df_ptdf_adj['line_id'] = None

for index, row in df_grid.iterrows():
    if row['tso'] == 'TENNET NL':
        df_grid.loc[index, 'tso'] = 'TENNETBV'
    if row['tso'] == 'Amprion GmbH':
        df_grid.loc[index, 'tso'] = 'AMPRION'
    if row['tso'] == 'TEL':
        df_grid.loc[index, 'tso'] = 'TRANSELECTRICA'

for eic in unique_eics:
    line_name = df_grid[df_grid.eic == eic].iloc[0]['name']
    print(line_name)
    line_id = df_grid[df_grid.eic == eic].index[0]
    df_ptdf_adj.loc[df_ptdf_adj.eic == eic, 'line_id'] = line_id
    df_ptdf_adj.loc[
        (df_ptdf_adj.eic == eic) &
        (line_name[:2] != df_ptdf_adj.name.str[:2])
    , hubs] = -1 * df_ptdf_adj.loc[
        (df_ptdf_adj.eic == eic) &
        (line_name[:2] != df_ptdf_adj.name.str[:2])
    , hubs]

df_ptdf_non_unique_eic = df_ptdf[df_ptdf.eic.isin(non_unique_eics)].groupby(['eic', 'direction', 'tso']).head(1).sort_values(by=['eic'])

identified_pair_eic = []

identified_tielines_eic = []
tielines = []
remainder_eics = []

for eic in non_unique_eics:
    df_ptdf_nu_eic = df_ptdf_non_unique_eic[df_ptdf_non_unique_eic.eic == eic]
    
    for tso in df_ptdf_nu_eic.tso.unique():
        df_pair = df_ptdf_nu_eic[(df_ptdf_nu_eic.tso == tso)]
        df_pair_direct = df_pair[df_pair.direction == 'DIRECT']
        df_pair_opposite = df_pair[df_pair.direction == 'OPPOSITE']
        
        df_line = df_grid[(df_grid.eic == eic) & (df_grid.tso == tso)]
        if len(df_line) == 2: # duplicate because of two directions
            identified_pair_eic.append(eic)
            if df_line.iloc[0]['name'][:2] == df_pair_direct.iloc[0]['name'][:2]:
                df_ptdf_adj.loc[
                    (df_ptdf_adj.eic == eic) & 
                    (df_ptdf_adj.tso == tso) & 
                    (df_ptdf_adj.direction == 'DIRECT'), 'line_id'] = df_line.index[0]
            else:
                df_ptdf_adj.loc[
                    (df_ptdf_adj.eic == eic) & 
                    (df_ptdf_adj.tso == tso) & 
                    (df_ptdf_adj.direction == 'OPPOSITE'), 'line_id'] = df_line.index[0] 

            if df_line.iloc[1]['name'][:2] == df_pair_direct.iloc[0]['name'][:2]:
                df_ptdf_adj.loc[
                    (df_ptdf_adj.eic == eic) & 
                    (df_ptdf_adj.tso == tso) & 
                    (df_ptdf_adj.direction == 'DIRECT'), 'line_id'] = df_line.index[1]
            else:
                df_ptdf_adj.loc[
                    (df_ptdf_adj.eic == eic) & 
                    (df_ptdf_adj.tso == tso) & 
                    (df_ptdf_adj.direction == 'OPPOSITE'), 'line_id'] = df_line.index[1] 
        
        elif len(df_line) == 1: # it was duplicate because of two TSOs
            identified_tielines_eic.append(eic)
            df_ptdf_adj.loc[(df_ptdf_adj.eic == eic) & (df_ptdf_adj.tso == tso), 'line_id'] = df_line.index[0]

        else:
            remainder_eics.append(eic)
            
print('Identified pair EICs: ', len(identified_pair_eic))
print('Identified tieline EICs: ', len(identified_tielines_eic))
df_ptdf_non_unique_eic = df_ptdf_non_unique_eic[df_ptdf_non_unique_eic.eic.isin(remainder_eics)]

display(df_ptdf_non_unique_eic)

#for index, eic in enumerate(remainder_eic):
#    df_ptdf_adj.loc[(df_ptdf_adj.eic == eic) & (df_ptdf_adj.tso == remainder_tso[index]) & (df_ptdf_adj.direction == remainder_direction[index]), 'line_id'] = remainder_line_ids[index]
    
display(df_ptdf_adj[df_ptdf_adj.line_id.isna()])

Total: 372
Unique: 202
Non-valid observed eics: 77


C:\Users\balaz\AppData\Local\Temp\ipykernel_12004\3132903498.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ptdf_adj['line_id'] = None


Ernsthofen - Hausruck 203C
Sarasdorf - Zurndorf 439A
ACHENE – GRAMME 380.10
GRAMME – LIXHE 380.11
LIXHE – VAN EYCK 380.91
Y-Simbach (-St. Peter - Altheim) 233/230
Mittelbexbach - Uchtelfangen BLIES S 
Schmölln - Dresden/Süd 555
Vierraden - Pasewalk 306
Gonyu - Gyor
Gyor - Oroszlany
Felsozsolca - Sajoivanka
RZESZOW-KROSNO ISKRZYNIA
Paroseni - Targiu Jiu Nord
Gabcikovo - P. Biskupice
H. Zdana - Sucany
V. Dur - Krizovany
V. Dur - Levice
Y-Simbach (-St. Peter - Altheim) 234/230
Paffendorf - Rommerskirchen PAFFEN S 
St.Peter - Altheim 233/230
Westtirol - Silz 414
Y-Oberzier (-Paffendorf - Sechtem) SECHTM S 
Liter - Martonvasar
Krimpen a/d IJssel-Geertruidenberg 380 W
V. Dur - Levice
Resita - Timisoara c1
Portile de Fier - Resita c1
Pulgar - Vieselbach 589
Dunamenti - Oroszlany
Borssele-Rilland 380 G
Diemen-Lelystad 380 W
Krimpen a/d IJssel-Breukelen Kortrijk 380 W
Oostzaan-Diemen 380 G
Hessenberg - Weissenbach 223
Altheim - Sittling 220
Lelystad-Ens 380 W
Ens-Zwolle 380 W
LIT 400kV N0 1 AVE

,DateTime,name,eic,tso,direction,substation_from,substation_to,ram,ALBE,ALDE,AT,BE,CZ,DE_LU,FR,HR,HU,NL,PL,RO,SI,SK
218013,2022-11-24 08:00:00,Y-Mercator (-Doel - Lillo) 380.51,22T-BE-IN-LI014Z,ELIA,OPPOSITE,Mercator,Liefkenshoek,1015.0,-0.07687,0.04123,-0.00579,-0.17680,0.00750,0.02260,-0.08939,-0.00402,0.00144,0.16344,0.00968,0.00047,-0.00740,0.00379
373991,2023-01-24 03:00:00,Y-Doel (-Lillo - Mercator) 380.52,22T-BE-IN-LI015X,ELIA,DIRECT,Doel,Liefkenshoek,951.0,-0.08411,0.02233,-0.01350,-0.16905,-0.00192,0.00997,-0.08757,-0.01150,-0.00705,0.17886,0.00027,-0.00785,-0.01418,-0.00489
626,2022-09-01 05:00:00,Y-Mercator (-Doel - Lillo) 380.52,22T-BE-IN-LI015X,ELIA,OPPOSITE,Mercator,Kallo,651.0,-0.05140,0.01479,-0.00286,-0.14117,0.00384,0.00921,-0.03562,-0.00177,0.00052,0.06122,0.00452,0.00016,-0.00307,0.00164
39026,2022-09-16 12:00:00,Y-Gramme (-Courcelles - Tergnee) 380.31,22T-BE-IN-LI019P,ELIA,DIRECT,Gramme,Saint Amand,308.0,0.08956,-0.00906,0.00203,-0.04200,-0.00146,-0.00447,0.02067,0.00157,0.00010,-0.03743,-0.00205,0.00026,0.00226,-0.00050
217694,2022-11-24 05:00:00,Y-Mercator (-Horta - Rodenhuize) 380.73,22T20161020---2C,ELIA,OPPOSITE,Mercator,Rodenhuize,1525.0,-0.08971,-0.05940,0.00608,0.02369,-0.00952,-0.02574,0.12815,0.00480,-0.00169,-0.12351,-0.01183,-0.00037,0.00807,-0.00472
210750,2022-11-21 14:00:00,Y-Horta (-Mercator - Rodenhuize) 380.73,22T20161020---2C,ELIA,DIRECT,Horta,Rodenhuize,1428.0,-0.08927,-0.05794,0.00144,-0.04404,-0.00920,-0.02621,0.11675,0.00439,-0.00164,-0.11337,-0.01120,-0.00041,0.00760,-0.00474
198065,2022-11-16 20:00:00,Y-Mercator (-Horta - Rodenhuize) 380.74,22T20161020---3A,ELIA,OPPOSITE,Mercator,Rodenhuize,1512.0,-0.08414,-0.05675,0.00644,0.02745,-0.00991,-0.02016,0.12591,0.00469,-0.00183,-0.11558,-0.01201,-0.00055,0.00797,-0.00499
429540,2023-02-15 05:00:00,Y-Mercator (-Horta - Rodenhuize) 380.74,22T20161020---3A,ELIA,DIRECT,Mercator,Rodenhuize,3.0,0.15806,0.07222,-0.00759,0.03349,0.01144,0.03014,-0.14756,-0.00625,0.00214,0.13855,0.01385,0.00043,-0.01006,0.00560
267,2022-09-01 02:00:00,Y-Van Eyck (-Andre Dumont - Gramme) 380.12,22T20161020---48,ELIA,OPPOSITE,Van Eyck,Zuttendaal,1004.0,-0.09548,-0.03370,0.00127,0.20473,-0.00572,-0.01158,0.04929,0.00286,-0.00100,-0.03735,-0.00645,-0.00023,0.00357,-0.00264
18,2022-09-01 00:00:00,Y-Gramme (-Andre Dumont - Van Eyck) 380.12,22T20161020---48,ELIA,DIRECT,Gramme,Zuttendaal,995.0,-0.11957,-0.03186,0.00123,0.19074,-0.00547,-0.01111,0.04717,0.00272,-0.00096,-0.03748,-0.00620,-0.00022,0.00341,-0.00254


,DateTime,name,eic,tso,direction,substation_from,substation_to,ram,ALBE,ALDE,AT,BE,CZ,DE_LU,FR,HR,HU,NL,PL,RO,SI,SK,line_id
18,2022-09-01 00:00:00,Y-Gramme (-Andre Dumont - Van Eyck) 380.12,22T20161020---48,ELIA,DIRECT,Gramme,Zuttendaal,995.0,-0.11957,-0.03186,0.00123,0.19074,-0.00547,-0.01111,0.04717,0.00272,-0.00096,-0.03748,-0.00620,-0.00022,0.00341,-0.00254,None
139,2022-09-01 01:00:00,Y-Gramme (-Andre Dumont - Van Eyck) 380.12,22T20161020---48,ELIA,DIRECT,Gramme,Zuttendaal,989.0,-0.11878,-0.03211,0.00125,0.19328,-0.00556,-0.01130,0.04783,0.00277,-0.00097,-0.03926,-0.00630,-0.00022,0.00347,-0.00259,None
267,2022-09-01 02:00:00,Y-Van Eyck (-Andre Dumont - Gramme) 380.12,22T20161020---48,ELIA,OPPOSITE,Van Eyck,Zuttendaal,1004.0,-0.09548,-0.03370,0.00127,0.20473,-0.00572,-0.01158,0.04929,0.00286,-0.00100,-0.03735,-0.00645,-0.00023,0.00357,-0.00264,None
384,2022-09-01 03:00:00,Y-Van Eyck (-Andre Dumont - Gramme) 380.12,22T20161020---48,ELIA,OPPOSITE,Van Eyck,Zuttendaal,1012.0,-0.09549,-0.03371,0.00127,0.20472,-0.00572,-0.01160,0.04914,0.00285,-0.00100,-0.03741,-0.00643,-0.00023,0.00356,-0.00251,None
508,2022-09-01 04:00:00,Y-Van Eyck (-Andre Dumont - Gramme) 380.12,22T20161020---48,ELIA,OPPOSITE,Van Eyck,Zuttendaal,1077.0,-0.09544,-0.03366,0.00128,0.20477,-0.00572,-0.01267,0.04945,0.00282,-0.00098,-0.03934,-0.00638,-0.00024,0.00358,-0.00251,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
534679,2023-03-29 04:00:00,Y-Gramme (-Andre Dumont - Van Eyck) 380.12,22T20161020---48,ELIA,DIRECT,Gramme,Zuttendaal,1035.0,-0.05135,-0.04097,0.00298,0.15441,-0.00502,-0.01543,0.05996,0.00252,-0.00101,-0.04018,-0.00831,-0.00037,0.00468,-0.00266,None
534898,2023-03-29 06:00:00,Y-Gramme (-Andre Dumont - Van Eyck) 380.12,22T20161020---48,ELIA,DIRECT,Gramme,Zuttendaal,1065.0,-0.05337,-0.04449,0.00368,0.15331,-0.00549,-0.01734,0.06111,0.00315,-0.00127,-0.03530,-0.00885,-0.00044,0.00523,-0.00302,None
534899,2023-03-29 06:00:00,Y-Gramme (-Andre Dumont - Van Eyck) 380.12,22T20161020---48,ELIA,DIRECT,Gramme,Zuttendaal,1085.0,0.22871,-0.04848,0.00402,0.17090,-0.00597,-0.01888,0.06619,0.00342,-0.00138,-0.03848,-0.00963,-0.00048,0.00569,-0.00329,None
535098,2023-03-29 08:00:00,Y-Gramme (-Andre Dumont - Van Eyck) 380.12,22T20161020---48,ELIA,DIRECT,Gramme,Zuttendaal,1078.0,-0.05211,-0.04490,0.00424,0.15821,-0.00555,-0.01760,0.06239,0.00336,-0.00130,-0.03684,-0.00894,-0.00045,0.00531,-0.00303,None


In [12]:
remainder_eic = [
    '22T-BE-IN-LI014Z', 
    '22T-BE-IN-LI015X', 
    '22T-BE-IN-LI019P',
    '22T20161020---2C',
    '22T20161020---3A', 
    '22T20161020---48',
]

#df_ptdf_remainder_combinations = df_ptdf[df_ptdf.eic.isin(remainder_eic)].groupby(['eic', 'direction', 'tso', 'substation_from', 'substation_to']).head(1).sort_values(by=['eic'])
#df_ptdf_remainder_combinations[['name', 'eic', 'tso', 'substation_from', 'substation_to']].to_excel('eic_manual_completion.xlsx')

In [23]:
df_ptdf_remainder_filled = pd.read_excel('eic_manual_completion.xlsx', index_col=0)

for index, row in df_ptdf_remainder_filled.iterrows():
    df_ptdf_adj.loc[
        (df_ptdf_adj.eic == row['eic']) & 
        (df_ptdf_adj.tso == row['tso']) & 
        (df_ptdf_adj.substation_from == row['substation_from']) &
        (df_ptdf_adj.substation_to == row['substation_to']), 'line_id'] = row['line_id']
    
df_ptdf_adj['direction'] = df_ptdf_adj['direction'].apply(lambda x: (1 if x == 'DIRECT' else -1))
df_ptdf_adj = df_ptdf_adj[['DateTime', 'line_id', 'direction', 'ram'] + hubs]

display(df_ptdf_adj)
#df_ptdf_adj.to_excel("ptdf_z.xlsx")

C:\Users\balaz\AppData\Local\Temp\ipykernel_12004\521093770.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ptdf_adj['direction'] = df_ptdf_adj['direction'].apply(lambda x: (1 if x == 'DIRECT' else -1))


,DateTime,line_id,direction,ram,ALBE,ALDE,AT,BE,CZ,DE_LU,FR,HR,HU,NL,PL,RO,SI,SK
0,2022-09-01 00:00:00,493,1,439.0,-0.06186,-0.06238,-0.06499,-0.06181,-0.02490,-0.06303,-0.06025,-0.00273,0.00340,-0.06144,-0.02183,0.00010,-0.00259,-0.00200
1,2022-09-01 00:00:00,493,-1,341.0,0.06186,0.06238,0.06499,0.06181,0.02490,0.06303,0.06025,0.00273,-0.00340,0.06144,0.02183,-0.00010,0.00259,0.00200
2,2022-09-01 00:00:00,2710,1,146.0,0.02502,0.02502,0.03672,0.02504,0.02301,0.02519,0.02522,0.01195,-0.01617,0.02482,0.01430,-0.00085,0.02081,0.00497
3,2022-09-01 00:00:00,2699,1,155.0,0.02473,0.02472,0.03740,0.02476,0.02248,0.02487,0.02501,0.01253,-0.01655,0.02451,0.01386,-0.00089,0.02190,0.00451
4,2022-09-01 00:00:00,2699,-1,267.0,-0.02473,-0.02472,-0.03740,-0.02476,-0.02248,-0.02487,-0.02501,-0.01253,0.01655,-0.02451,-0.01386,0.00089,-0.02190,-0.00451
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
541475,2023-03-31 22:00:00,2036,1,410.0,0.16091,0.16134,0.16359,0.16079,0.18497,0.16210,0.15977,0.08526,0.03712,0.16120,0.12676,-0.00840,0.13222,0.12419
541476,2023-03-31 22:00:00,2814,1,1421.0,-0.12423,-0.12522,-0.10793,-0.12392,-0.18969,-0.12661,-0.12154,-0.03307,0.01876,-0.12507,-0.07989,0.00276,-0.06246,0.06681
541477,2023-03-31 22:00:00,2815,-1,1004.0,0.11961,0.12068,0.09994,0.11927,0.18129,0.12210,0.11665,0.02803,-0.02225,0.12060,0.08492,-0.00214,0.05525,-0.07919
541479,2023-03-31 22:00:00,2813,-1,1179.0,0.15186,0.15387,0.09884,0.15112,0.15771,0.15581,0.14591,0.03331,-0.00911,0.15457,0.19936,-0.00316,0.05968,-0.06298


In [24]:
df_ptdf_adj_reduced = df_ptdf_adj.copy()

df_ptdf_adj_reduced.loc[df_ptdf_adj_reduced.direction == -1, hubs] = -1 * df_ptdf_adj_reduced.loc[df_ptdf_adj_reduced.direction == -1, hubs]

df_ptdf_adj_reduced = df_ptdf_adj_reduced[['DateTime', 'line_id', 'ram'] + hubs].groupby(['DateTime', 'line_id'], as_index=False).mean()

display(df_ptdf_adj_reduced)
#df_ptdf_adj_reduced.to_excel("ptdf_z_reduced.xlsx")


,DateTime,line_id,ram,ALBE,ALDE,AT,BE,CZ,DE_LU,FR,HR,HU,NL,PL,RO,SI,SK
0,2022-09-01 00:00:00,12,153.0,-0.048360,-0.048610,-0.01764,-0.047910,-0.012870,-0.045900,-0.042440,-0.007560,0.000850,-0.051740,0.042200,0.000710,-0.01033,0.007150
1,2022-09-01 00:00:00,94,653.0,-0.019600,-0.020160,-0.02146,-0.019940,-0.072370,-0.037740,-0.023400,-0.007990,-0.000630,-0.015850,0.082020,0.000820,-0.01122,0.007690
2,2022-09-01 00:00:00,152,1140.0,-0.123610,-0.090080,0.00777,-0.134080,0.006720,-0.002220,-0.140700,-0.003950,0.001760,-0.071220,0.003640,0.000300,-0.00444,0.003040
3,2022-09-01 00:00:00,223,919.0,-0.088810,-0.116440,0.01279,-0.082860,-0.021460,-0.007370,0.006380,0.009440,-0.003090,-0.097610,-0.025080,-0.000780,0.01233,-0.009380
4,2022-09-01 00:00:00,333,887.0,0.083420,0.081460,-0.01833,0.084800,-0.006460,0.058030,0.097120,0.007620,-0.004400,0.075000,0.005170,-0.000530,0.00870,-0.004570
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
264955,2023-03-31 22:00:00,2832,1097.0,0.071407,0.071837,0.03525,0.071433,-0.140247,0.069737,0.070987,0.008713,-0.003793,0.070440,0.005797,-0.000803,0.01641,-0.009690
264956,2023-03-31 22:00:00,2833,983.0,0.081680,0.081360,0.05098,0.082130,-0.072190,0.075710,0.084070,0.012850,-0.004040,0.078440,-0.009400,-0.001240,0.02356,-0.014210
264957,2023-03-31 22:00:00,2842,262.0,0.047490,0.048630,-0.01230,0.047200,0.015300,0.046590,0.044950,-0.000540,-0.001840,0.048000,0.018920,0.000110,-0.00013,0.002490
264958,2023-03-31 22:00:00,2844,262.0,0.047480,0.048620,-0.01230,0.047200,0.015300,0.046580,0.044950,-0.000540,-0.001840,0.048000,0.018920,0.000110,-0.00013,0.002490


In [25]:
df_ptdf_adj_reduced.to_excel("ptdf_z_reduced.xlsx")